In [2]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
import datetime as dt
from datetime import timedelta, datetime

In [3]:
# read in the cohort file
cohort = pd.read_parquet(os.getcwd() + '/first_dementia_dx_all.parquet')

In [4]:
cohort.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care
0,792199168,2010-11-25,1.0
1,24117254,2005-10-04,1.0
2,519045131,2004-02-11,0.0
3,63438860,1996-09-26,1.0
4,734003216,2009-08-26,1.0


In [5]:
#read in the patient data and format the date columns

patient = pd.read_csv('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/19_050_gold_extract_patient_001.txt', sep="\t")
patient['frd']= patient['frd'].str.replace('/', '')
patient['frd'] = pd.to_datetime(patient['frd'], format='%d%m%Y')
patient['crd']= patient['crd'].str.replace('/', '')
patient['crd'] = pd.to_datetime(patient['crd'], format='%d%m%Y')
patient['tod']= patient['tod'].str.replace('/', '')
patient['tod'] = pd.to_datetime(patient['tod'], format='%d%m%Y')
patient['deathdate']= patient['deathdate'].str.replace('/', '')
patient['deathdate'] = pd.to_datetime(patient['deathdate'], format='%d%m%Y')
#drop the duplicate values for e_patid in the patient file
patient = patient.drop_duplicates(subset='e_patid', keep=False)

In [6]:
# save formatted patient file
patient.to_parquet(os.getcwd() + '/formatted_patient_file.parquet')

In [7]:
#read in deathdate from linked data
deathdata = pd.read_csv("/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Linked Data/gold_death_patient_19_050.txt",  sep="\t", usecols =(['e_patid', 'dod', 'dod_partial']),  dtype = ({'dod_partial':'str'}))
deathdata['dod']= deathdata['dod'].str.replace('/', '')
deathdata['dod'] = pd.to_datetime(deathdata['dod'], format='%d%m%Y')

In [8]:
cohort_e_patids = list(cohort['e_patid'])

In [9]:
# merge in death data
cohort_patient_df = patient[patient['e_patid'].isin(cohort_e_patids)].merge(
    deathdata[['e_patid', 'dod', 'dod_partial']], how="left", on="e_patid")

In [10]:
#create a merged death date variable. Uses ONS data if avaiable, but cprd data if ons data is missing
cohort_patient_df['dod'].fillna(cohort_patient_df['deathdate'], inplace=True)

In [11]:
cohort_patient_df.head()

,e_patid,gender,yob,marital,frd,crd,regstat,reggap,internal,tod,toreason,deathdate,accept,dod,dod_partial
0,100004237,1,1929,6,1988-04-28,1988-04-28,0,0,0,2008-09-10,1,2008-09-08,1,2008-09-08,NaN
1,100015146,1,1935,6,1984-06-06,1984-06-06,0,0,0,NaT,0,NaT,1,NaT,NaN
2,10002112,1,1930,6,1980-03-25,1980-03-25,0,0,0,2005-05-26,1,2005-04-08,1,2005-04-08,NaN
3,100024196,2,1934,0,1999-09-17,1999-09-17,0,0,0,2010-04-16,27,NaT,1,NaT,NaN
4,100025100,2,1916,6,1985-04-30,1985-04-30,0,0,0,2010-09-01,27,NaT,1,2012-06-02,NaN


In [12]:
# save raw file with death data
cohort_patient_df.to_parquet(os.getcwd() + '/raw_cohort_patient_file.parquet')

In [13]:
cohort_patient_df.describe()

,e_patid,gender,yob,marital,regstat,reggap,internal,toreason,accept
count,1.145760e+05,114576.000000,114576.000000,114576.000000,114576.000000,114576.000000,114576.000000,114576.000000,114576.0
mean,2.730966e+08,1.651620,1925.517901,3.727133,0.015632,29.438774,0.017255,5.170036,1.0
std,2.551660e+08,0.476459,8.846132,2.635836,0.134510,322.685707,0.137649,9.195810,0.0
min,2.888000e+03,1.000000,1894.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,4.537418e+07,1.000000,1920.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.842646e+08,2.000000,1925.000000,6.000000,0.000000,0.000000,0.000000,1.000000,1.0
75%,4.927444e+08,2.000000,1931.000000,6.000000,0.000000,0.000000,0.000000,3.000000,1.0
max,8.000031e+08,2.000000,1954.000000,11.000000,6.000000,16061.000000,4.000000,34.000000,1.0


In [14]:
# copy data to raw file
raw_cohort_data = cohort.copy(deep=True)
raw_cohort_data['gender'] = cohort_patient_df['gender']
raw_cohort_data['yob'] = cohort_patient_df['yob']
raw_cohort_data['frd'] = cohort_patient_df['frd']
raw_cohort_data['crd'] = cohort_patient_df['crd']
raw_cohort_data['tod'] = cohort_patient_df['tod']
raw_cohort_data['dod'] = cohort_patient_df['dod']

In [15]:
raw_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod
0,792199168,2010-11-25,1.0,1.0,1929.0,1988-04-28,1988-04-28,2008-09-10,2008-09-08
1,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT
2,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08
3,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT
4,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02


In [16]:
# drop nans
cleaner_cohort_data = raw_cohort_data.dropna(subset=['gender', 'yob'])

In [17]:
cleaner_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod
0,792199168,2010-11-25,1.0,1.0,1929.0,1988-04-28,1988-04-28,2008-09-10,2008-09-08
1,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT
2,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08
3,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT
4,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02


In [18]:
cleaner_cohort_data.shape[0]

114576

In [19]:
cleaner_cohort_data.describe()

,e_patid,first_diagnosed_in_primary_care,gender,yob
count,1.145760e+05,114576.000000,114576.000000,114576.000000
mean,3.106449e+08,0.248883,1.651620,1925.517901
std,2.548436e+08,0.432368,0.476459,8.846132
min,1.732000e+03,0.000000,1.000000,1894.000000
25%,6.086343e+07,0.000000,1.000000,1920.000000
50%,2.702427e+08,0.000000,2.000000,1925.000000
75%,5.348966e+08,0.000000,2.000000,1931.000000
max,7.998256e+08,1.000000,2.000000,1954.000000


In [20]:
# function to get min of tod and deathdate
def date_function(tod, deathdate):
    if tod is pd.NaT:
        if deathdate is pd.NaT:
            return pd.Timestamp('2022-12-31')
        else:
            return deathdate
    elif deathdate is pd.NaT:
        return tod
    else:
        return min(tod, deathdate)

In [21]:
#create min end date column
cleaner_cohort_data['min_end_date'] = cleaner_cohort_data.apply(lambda x: date_function(x.tod, x.dod), axis=1)

/mnt/ebinstall/software/Anaconda3/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
cleaner_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod,min_end_date
0,792199168,2010-11-25,1.0,1.0,1929.0,1988-04-28,1988-04-28,2008-09-10,2008-09-08,2008-09-08
1,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT,2022-12-31
2,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08,2005-04-08
3,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT,2010-04-16
4,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02,2010-09-01


In [27]:
# limit to diagnosis before min end date
even_cleaner_cohort_data = cleaner_cohort_data[cleaner_cohort_data['first_diagnosis_date'] < cleaner_cohort_data['min_end_date']]

In [28]:
even_cleaner_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod,min_end_date
1,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT,2022-12-31
2,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08,2005-04-08
3,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT,2010-04-16
4,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02,2010-09-01
8,510656537,2003-08-22,0.0,2.0,1914.0,1982-08-02,1982-08-02,2004-02-12,2004-01-15,2004-01-15


In [29]:
even_cleaner_cohort_data.shape[0]

64204

In [25]:
even_cleaner_cohort_data.describe()

,e_patid,first_diagnosed_in_primary_care,gender,yob
count,6.420400e+04,64204.000000,64204.000000,64204.000000
mean,3.092577e+08,0.293580,1.643137,1928.213009
std,2.546851e+08,0.455405,0.479077,8.366285
min,1.732000e+03,0.000000,1.000000,1894.000000
25%,6.055774e+07,0.000000,1.000000,1922.000000
50%,2.670417e+08,0.000000,2.000000,1928.000000
75%,5.333030e+08,1.000000,2.000000,1934.000000
max,7.997974e+08,1.000000,2.000000,1954.000000


In [26]:
even_cleaner_cohort_data.dtypes

e_patid                                     int64
first_diagnosis_date               datetime64[ns]
first_diagnosed_in_primary_care           float64
gender                                    float64
yob                                       float64
frd                                datetime64[ns]
crd                                datetime64[ns]
tod                                datetime64[ns]
dod                                datetime64[ns]
min_end_date                       datetime64[ns]
dtype: object

In [27]:
# make end year column
even_cleaner_cohort_data['end_year'] = even_cleaner_cohort_data['min_end_date'].dt.year

/mnt/ebinstall/software/Anaconda3/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
even_cleaner_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod,min_end_date,end_year
1,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT,2022-12-31,2022
2,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08,2005-04-08,2005
3,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT,2010-04-16,2010
4,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02,2010-09-01,2010
8,510656537,2003-08-22,0.0,2.0,1914.0,1982-08-02,1982-08-02,2004-02-12,2004-01-15,2004-01-15,2004


In [29]:
# make end year numeric
even_cleaner_cohort_data['end_year'] = pd.to_numeric(even_cleaner_cohort_data['end_year'])

/mnt/ebinstall/software/Anaconda3/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
even_cleaner_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod,min_end_date,end_year
1,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT,2022-12-31,2022
2,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08,2005-04-08,2005
3,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT,2010-04-16,2010
4,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02,2010-09-01,2010
8,510656537,2003-08-22,0.0,2.0,1914.0,1982-08-02,1982-08-02,2004-02-12,2004-01-15,2004-01-15,2004


In [31]:
even_cleaner_cohort_data.dtypes

e_patid                                     int64
first_diagnosis_date               datetime64[ns]
first_diagnosed_in_primary_care           float64
gender                                    float64
yob                                       float64
frd                                datetime64[ns]
crd                                datetime64[ns]
tod                                datetime64[ns]
dod                                datetime64[ns]
min_end_date                       datetime64[ns]
end_year                                    int64
dtype: object

In [32]:
# limit data to age at end year to be within 35-110 years old and birth after 1900 and create age_at_dx column
cleanest_cohort_data = even_cleaner_cohort_data[
    even_cleaner_cohort_data['end_year'] - even_cleaner_cohort_data['yob'] <= 110.0]
cleanest_cohort_data = cleanest_cohort_data[cleanest_cohort_data['end_year'] - cleanest_cohort_data['yob'] >= 35.0]
cleanest_cohort_data = cleanest_cohort_data[cleanest_cohort_data['yob'] >= 1900.0]
cleanest_cohort_data['age_at_dx'] = cleanest_cohort_data['first_diagnosis_date'].dt.year - cleanest_cohort_data['yob']

In [33]:
#read in the cohort file data
cohort_file = pd.read_csv("/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/19_050_gold_cohort.txt", sep="\t")

#drop the duplicate values for e_patid in the cohort file
cohort_file = cohort_file.drop_duplicates(subset='e_patid', keep=False)

In [34]:
#read in the practice data - need to format the date columns lcd and uts

practice = pd.read_csv("/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/19_050_gold_extract_practice_001.txt", sep="\t")
practice['lcd']= practice['lcd'].str.replace('/', '')
practice['lcd'] = pd.to_datetime(practice['lcd'], format='%d%m%Y')
practice['uts']= practice['uts'].str.replace('/', '')
practice['uts'] = pd.to_datetime(practice['uts'], format='%d%m%Y')

In [35]:
#read in the practice imd data
imd = pd.read_csv("/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Linked Data/gold_practice_imd_19_050.txt",  sep="\t")

In [36]:
# merge in e_pracid from the cohort file
cleanest_cohort_data = pd.merge(cleanest_cohort_data, cohort_file[['e_patid', 'e_pracid']], how="left", on="e_patid")

#merge in uts and region date from practice file 
cleanest_cohort_data = pd.merge(cleanest_cohort_data, practice[['e_pracid', 'region', 'uts']], how="left", on='e_pracid')

#merge in imd data from imd file
cleanest_cohort_data = pd.merge(cleanest_cohort_data, imd[['e_pracid', 'e2015_imd_5']], how="left", on="e_pracid")

In [37]:
cleanest_cohort_data.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,yob,frd,crd,tod,dod,min_end_date,end_year,age_at_dx,e_pracid,region,uts,e2015_imd_5
0,24117254,2005-10-04,1.0,1.0,1935.0,1984-06-06,1984-06-06,NaT,NaT,2022-12-31,2022,70.0,54.0,10.0,2000-01-12,4.0
1,519045131,2004-02-11,0.0,1.0,1930.0,1980-03-25,1980-03-25,2005-05-26,2005-04-08,2005-04-08,2005,74.0,131.0,2.0,1992-12-21,5.0
2,63438860,1996-09-26,1.0,2.0,1934.0,1999-09-17,1999-09-17,2010-04-16,NaT,2010-04-16,2010,62.0,60.0,8.0,1992-08-20,1.0
3,734003216,2009-08-26,1.0,2.0,1916.0,1985-04-30,1985-04-30,2010-09-01,2012-06-02,2010-09-01,2010,93.0,216.0,10.0,2003-02-14,4.0
4,510656537,2003-08-22,0.0,2.0,1914.0,1982-08-02,1982-08-02,2004-02-12,2004-01-15,2004-01-15,2004,89.0,NaN,NaN,NaT,NaN


In [38]:
#create markers 6, 12, 18, 24 months and 3y and 5y before diagnosis date in cleanest_cohort_data

cleanest_cohort_data["dx_minus6m"] = cleanest_cohort_data['first_diagnosis_date'] - timedelta(days = 183)
cleanest_cohort_data["dx_minus12m"] = cleanest_cohort_data['first_diagnosis_date'] - timedelta(days = 365)
cleanest_cohort_data["dx_minus18m"] = cleanest_cohort_data['first_diagnosis_date'] - timedelta(days = 548)
cleanest_cohort_data["dx_minus24m"] = cleanest_cohort_data['first_diagnosis_date'] - timedelta(days = 730)
cleanest_cohort_data["dx_minus3y"] = cleanest_cohort_data['first_diagnosis_date'] - timedelta(days = 1095)
cleanest_cohort_data["dx_minus5y"] = cleanest_cohort_data['first_diagnosis_date'] - timedelta(days = 1826)

# select only cases where uts date is at least a year before diagnosis date
cleanest_cohort_data = cleanest_cohort_data[cleanest_cohort_data.dx_minus12m >= cleanest_cohort_data.uts]

#select only cases with at least one year of data before lc diagnosis data
#have taken this to be cases where current registration date (crd) is at least one year before dx date
cleanest_cohort_data = cleanest_cohort_data[cleanest_cohort_data.dx_minus12m >= cleanest_cohort_data.crd]

In [39]:
cleanest_cohort_data.shape[0]

25453

In [40]:
cleanest_cohort_data.describe()

,e_patid,first_diagnosed_in_primary_care,gender,yob,end_year,age_at_dx,e_pracid,region,e2015_imd_5
count,2.545300e+04,25453.000000,25453.000000,25453.000000,25453.000000,25453.000000,25453.000000,25453.000000,25180.000000
mean,2.757523e+08,0.470121,1.637410,1928.494126,2014.992339,79.239932,134.882293,6.183515,3.313423
std,2.550430e+08,0.499116,0.480757,8.191687,5.540493,8.576975,75.199745,2.585470,1.381720
min,2.888000e+03,0.000000,1.000000,1900.000000,1998.000000,42.000000,1.000000,1.000000,1.000000
25%,4.638684e+07,0.000000,1.000000,1923.000000,2011.000000,74.000000,70.000000,4.000000,2.000000
50%,1.909682e+08,0.000000,2.000000,1928.000000,2015.000000,80.000000,138.000000,7.000000,4.000000
75%,4.966802e+08,1.000000,2.000000,1934.000000,2022.000000,85.000000,204.000000,8.000000,5.000000
max,7.997942e+08,1.000000,2.000000,1954.000000,2022.000000,108.000000,257.000000,10.000000,5.000000


In [41]:
# select columns to output
cohort_out = cleanest_cohort_data[['e_patid', 'first_diagnosis_date', 'first_diagnosed_in_primary_care', 
                                   'gender', 'age_at_dx', 'e_pracid', 'region', 'uts', 'e2015_imd_5', 'dx_minus6m',
                                  'dx_minus12m', 'dx_minus18m', 'dx_minus24m', 'dx_minus3y', 'dx_minus5y']]

In [42]:
cohort_out.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11
3,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26
7,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02
8,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15


In [43]:
# save to parquet
cohort_out.to_parquet(os.getcwd() + '/cleaned_cohort_file.parquet')